In [ ]:
# Install Necessary libraries

In [ ]:
!pip install pyngrok
!pip install -q openai
!pip install flask

In [ ]:
# Import libraries

In [ ]:
from flask import Flask, render_template, request
from pyngrok import ngrok
import os
import openai
import urllib.parse
import json

In [ ]:
# Setting up the google drive path where the project folder is located

In [ ]:
RESEARCH_DIR = "/content/drive/MyDrive/Final_tshirt_print_generation_dalle2_openapi"
os.chdir(RESEARCH_DIR)
os.getcwd()

'/content/drive/MyDrive/Final_tshirt_print_generation_dalle2_openapi'

In [ ]:
# Setting up the port number for ngrok

In [ ]:
port_no = 5000

In [ ]:
# Set up your OpenAI API credentials

In [ ]:
with open('OPEN_API_SECRET_KEY.json') as key:
  open_api_secret_key = json.load(key)

openai.api_key = open_api_secret_key['api_key']

In [ ]:
# Main script

In [ ]:
app = Flask(__name__, template_folder='./templates')

#set up ngrok token
with open('NGROK_AUTH_TOKEN.json') as token:
  ngrok_auth_token = json.load(token)

ngrok.set_auth_token(ngrok_auth_token['auth_token'])
public_url =  ngrok.connect(port_no).public_url


@app.route("/")
def home():
    return render_template('index_main.html')

@app.route('/show_products',methods=['POST','GET']) # route to show the products in a web UI
def show_product():
  if request.method == 'POST':
    return render_template('products.html')
  else:
    return "something went wrong"

@app.route('/black_tshirt',methods=['POST','GET'])
def black_tshirts():
  if request.method == 'POST':
    return render_template('detail_black.html')
  else:
    return "something went wrong"

@app.route('/white_tshirt',methods=['POST','GET'])
def white_tshirts():
  if request.method == 'POST':
    return render_template('detail_white.html')
  else:
    return "something went wrong"

@app.route('/yellow_tshirt',methods=['POST','GET'])
def yellow_tshirts():
  if request.method == 'POST':
    return render_template('detail_yellow.html')
  else:
    return "something went wrong"

@app.route('/generate_black_tshirt',methods=['POST','GET'])
def generate_black_tshirt():
  if request.method == 'POST':
    text_for_generation = str(request.form['text_to_generate'])
    response = openai.Image.create_edit(image=open("./static/black_512_512.png", "rb"),mask=open("./static/black_mask_512.png", "rb"),prompt=text_for_generation,n=1,size="512x512")
    image_url = response['data'][0]['url']

    # Encode the URL
    encoded_image_url = urllib.parse.quote(image_url, safe=':/?=&%')

    return render_template('generated_black_image.html',image_url=encoded_image_url)
  else:
    return "something went wrong"


@app.route('/generate_white_tshirt',methods=['POST','GET'])
def generate_white_tshirt():
  if request.method == 'POST':
    text_for_generation = str(request.form['text_to_generate'])
    response = openai.Image.create_edit(image=open("./static/white_512_512.png", "rb"),mask=open("./static/white_mask_512.png", "rb"),prompt=text_for_generation,n=1,size="512x512")
    image_url = response['data'][0]['url']

    #Encode the URL
    encoded_image_url = urllib.parse.quote(image_url, safe=':/?=&%')

    return render_template('generated_white_image.html',image_url=encoded_image_url)
  else:
    return "something went wrong"


@app.route('/generate_yellow_tshirt',methods=['POST','GET'])
def generate_yellow_tshirt():
  if request.method == 'POST':
    text_for_generation = str(request.form['text_to_generate'])
    response = openai.Image.create_edit(image=open("./static/yellow_512_512.png", "rb"),mask=open("./static/yellow_mask_512.png", "rb"),prompt=text_for_generation,n=1,size="512x512")
    image_url = response['data'][0]['url']

    #Encode the URL
    encoded_image_url = urllib.parse.quote(image_url, safe=':/?=&%')
    return render_template('generated_yellow_image.html',image_url=encoded_image_url)
  else:
    return "something went wrong"

print(f"To acces the Gloable link please click {public_url}")

#Runnng the App
app.run(port=port_no)
